# Personalized Student Flow Visualization

This notebook visualizes the estimated probability of a student choosing different destinations based on the four regression models.

## Models
- **Model 1**: Base Gravity Model
- **Model 2**: Origin Fixed Effects Model (High Cost = High Quality Signal)
- **Model 3**: Restricted Gravity Model (ROI Focused)
- **Model 4**: Parsimonious Gravity Model (Structural Baseline)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import HTML

# Load the predictions
df = pd.read_csv("../Datasets/personalized_predictions.csv")

# Display the data
display(df.head())

,destination_country_code,pred_m1_count,pred_m1_prob,pred_m2_count,pred_m2_prob,pred_m3_count,pred_m3_prob,pred_m4_count,pred_m4_prob
0,USA,256.658268,0.091951,2199.132841,0.158166,387.077732,0.066352,852.891962,0.084101
1,FRA,166.921488,0.059802,797.754063,0.057376,353.803188,0.060648,558.249702,0.055048
2,ARE,228.607312,0.081901,1548.460638,0.111369,435.004044,0.074568,766.565697,0.075589
3,CAN,190.833068,0.068368,1106.136429,0.079556,362.627297,0.062161,708.121373,0.069826
4,CHE,245.395481,0.087916,1502.860678,0.108089,355.778919,0.060987,727.250632,0.071712


In [ ]:
# Create an interactive 2D map
# We will create a figure with a dropdown menu to switch between models

fig = go.Figure()

models = [
    ("pred_m1_prob", "Model 1 (Base)"),
    ("pred_m2_prob", "Model 2 (Origin FE)"),
    ("pred_m3_prob", "Model 3 (ROI)"),
    ("pred_m4_prob", "Model 4 (Parsimonious)")
]

# Add a trace for each model (initially only the first one is visible)
for col, name in models:
    fig.add_trace(go.Choropleth(
        locations=df['destination_country_code'],
        z=df[col],
        text=df['destination_country_code'],
        colorscale='Blues',
        autocolorscale=False,
        reversescale=False,
        marker_line_color='white', # Changed to white to blend in or look cleaner
        marker_line_width=0.5,
        colorbar_title="Probability",
        visible=(col == "pred_m1_prob"),
        name=name
    ))

# Create buttons for the dropdown menu
buttons = []
for i, (col, name) in enumerate(models):
    # Create a visibility list: [False, False, ..., True, ..., False]
    visibility = [False] * len(models)
    visibility[i] = True
    
    button = dict(
        label=name,
        method="update",
        args=[{"visible": visibility},
              {"title": f"Student Flow Probability: {name}"}]
    )
    buttons.append(button)

# Update layout with dropdown and 2D projection
fig.update_layout(
    title_text="Student Flow Probability: Model 1 (Base)",
    height=650,
    autosize=True,
    margin=dict(l=0, r=0, t=60, b=20),
    dragmode=False,
    geo=dict(
        showframe=False,
        showcoastlines=False, # Removed coastlines
        projection_type='natural earth'
    ),
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=0.98,
        xanchor="right",
        y=1.08,
        yanchor="top",
        pad=dict(r=10, t=0)
    )]
)

for col, name in models:
    fig.update_traces(selector=dict(name=name), hovertemplate="%{text}<br>Probability: %{z:.2%}")

html = fig.to_html(
    include_plotlyjs="cdn",
    full_html=False,
    config={
        "responsive": True,
        "displayModeBar": False,
        "scrollZoom": False,
        "doubleClick": False
    }
)

display(HTML(html))